## Generate QC table

- Add additional crispr samples to the QC table.
- Fill out the details to [geo-submission-metadata-v4](https://docs.google.com/spreadsheets/d/1Y_OIFHcAEtofVmIso5LPKX3YpU04i1W6/edit#gid=1074977823)

In [1]:
import os
import pandas as pd
from pathlib import Path
from glob import glob
import json
import numpy as np
from tqdm import tqdm
from basepair.utils import md5sum, flatten_list
from joblib import Parallel, delayed

Using TensorFlow backend.


In [2]:
def read_config(config_path):
    config = json.load(open(config_path))
    # Merge all chip_nexus.fastqs into a list.
    config['fastqs'] = [p for k, paths in config.items()
                        if k.startswith('chip_nexus.fastqs')
                        for p in paths]
    return {k: v for k, v in config.items() if not k.startswith('chip_nexus.fastqs')}


def read_metadata(metadata_path, add_qc=True):
    from basepair.utils import flatten

    if not os.path.exists(metadata_path):
        return {}
    metadata = json.load(open(metadata_path))
    metadata['chip_nexus.qc_json'] = metadata['outputs']['chip_nexus.qc_json']
    metadata['chip_nexus.report'] = metadata['outputs']['chip_nexus.report']
    metadata = {k: v for k, v in metadata.items()
                if k in ['status', 'workflowRoot', 'id', 'chip_nexus.qc_json', 'chip_nexus.report']}
    if add_qc:
        metadata = {**metadata, **flatten(json.load(open(metadata['chip_nexus.qc_json'])), '/')}
    return metadata

In [3]:
# Read configs
samples = [
    'wt.Nanog',
    'wt.Sox2',
    'mutant.sox2crispr1.Nanog',
    'mutant.sox2crispr1.Sox2',
    'mutant.nanogcrispr1.Nanog',
    'mutant.nanogcrispr1.Sox2'
]

# Load configs
ddir = Path('/oak/stanford/groups/akundaje/avsec/basepair/data/raw/zeitlinger-chip-nexus-crispr/')
configs = {sample: {**read_config(f'{ddir}/{sample}.config.json'),
                    **read_metadata(f'{ddir}/{sample}.metadata.json')}
           for sample in samples}

df = pd.DataFrame(configs).T

In [5]:
# Sanity checks
all_fastqs = flatten_list(df.fastqs)
# No duplicates in fastqs
assert not pd.Series(all_fastqs).duplicated().any()
# All fastq files were processed
# assert {str(s) for s in ddir.glob('*.fastq.gz')} == set(all_fastqs)

# All adapters and barcodes were the same for all experiments
assert len(df['chip_nexus.adapter'].unique()) == 1
assert len(df['chip_nexus.barcodes'].unique()) == 1

# All files succeeded
assert np.all(df['status'] == 'Succeeded')

In [6]:
df['chip_nexus.report']

wt.Nanog                     /oak/stanford/groups/...
wt.Sox2                      /oak/stanford/groups/...
mutant.sox2crispr1.Nanog     /oak/stanford/groups/...
mutant.sox2crispr1.Sox2      /oak/stanford/groups/...
mutant.nanogcrispr1.Nanog    /oak/stanford/groups/...
mutant.nanogcrispr1.Sox2     /oak/stanford/groups/...
Name: chip_nexus.report, dtype: object

In [7]:
df.head()

,chip_nexus.adapter,chip_nexus.barcodes,chip_nexus.description,chip_nexus.enable_count_signal_track,chip_nexus.genome_tsv,chip_nexus.qc_json,chip_nexus.report,chip_nexus.title,fastqs,flagstat_qc/rep1/diff_chroms,flagstat_qc/rep1/diff_chroms_qc_failed,flagstat_qc/rep1/duplicates,flagstat_qc/rep1/duplicates_qc_failed,flagstat_qc/rep1/mapped,flagstat_qc/rep1/mapped_pct,flagstat_qc/rep1/mapped_qc_failed,flagstat_qc/rep1/paired,flagstat_qc/rep1/paired_properly,flagstat_qc/rep1/paired_properly_pct,flagstat_qc/rep1/paired_properly_qc_failed,flagstat_qc/rep1/paired_qc_failed,flagstat_qc/rep1/read1,flagstat_qc/rep1/read1_qc_failed,flagstat_qc/rep1/read2,flagstat_qc/rep1/read2_qc_failed,flagstat_qc/rep1/singletons,flagstat_qc/rep1/singletons_pct,flagstat_qc/rep1/singletons_qc_failed,flagstat_qc/rep1/total,flagstat_qc/rep1/total_qc_failed,flagstat_qc/rep1/with_itself,flagstat_qc/rep1/with_itself_qc_failed,flagstat_qc/rep2/diff_chroms,flagstat_qc/rep2/diff_chroms_qc_failed,flagstat_qc/rep2/duplicates,flagstat_qc/rep2/duplicates_qc_failed,flagstat_qc/rep2/mapped,flagstat_qc/rep2/mapped_pct,flagstat_qc/rep2/mapped_qc_failed,flagstat_qc/rep2/paired,flagstat_qc/rep2/paired_properly,flagstat_qc/rep2/paired_properly_pct,flagstat_qc/rep2/paired_properly_qc_failed,flagstat_qc/rep2/paired_qc_failed,flagstat_qc/rep2/read1,flagstat_qc/rep2/read1_qc_failed,flagstat_qc/rep2/read2,flagstat_qc/rep2/read2_qc_failed,flagstat_qc/rep2/singletons,flagstat_qc/rep2/singletons_pct,flagstat_qc/rep2/singletons_qc_failed,flagstat_qc/rep2/total,flagstat_qc/rep2/total_qc_failed,flagstat_qc/rep2/with_itself,flagstat_qc/rep2/with_itself_qc_failed,flagstat_qc/rep3/diff_chroms,flagstat_qc/rep3/diff_chroms_qc_failed,flagstat_qc/rep3/duplicates,flagstat_qc/rep3/duplicates_qc_failed,flagstat_qc/rep3/mapped,flagstat_qc/rep3/mapped_pct,flagstat_qc/rep3/mapped_qc_failed,flagstat_qc/rep3/paired,flagstat_qc/rep3/paired_properly,flagstat_qc/rep3/paired_properly_pct,flagstat_qc/rep3/paired_properly_qc_failed,flagstat_qc/rep3/paired_qc_failed,flagstat_qc/rep3/read1,flagstat_qc/rep3/read1_qc_failed,flagstat_qc/rep3/read2,flagstat_qc/rep3/read2_qc_failed,flagstat_qc/rep3/singletons,flagstat_qc/rep3/singletons_pct,flagstat_qc/rep3/singletons_qc_failed,flagstat_qc/rep3/total,flagstat_qc/rep3/total_qc_failed,flagstat_qc/rep3/with_itself,flagstat_qc/rep3/with_itself_qc_failed,frip_macs2_qc/pooled/FRiP,frip_macs2_qc/ppr1/FRiP,frip_macs2_qc/ppr2/FRiP,frip_macs2_qc/rep1-pr1/FRiP,frip_macs2_qc/rep1-pr2/FRiP,frip_macs2_qc/rep1/FRiP,frip_macs2_qc/rep2-pr1/FRiP,frip_macs2_qc/rep2-pr2/FRiP,frip_macs2_qc/rep2/FRiP,frip_macs2_qc/rep3-pr1/FRiP,frip_macs2_qc/rep3-pr2/FRiP,frip_macs2_qc/rep3/FRiP,general/date,general/description,general/genome,general/paired_end,general/peak_caller,general/pipeline_ver,general/title,id,idr_frip_qc/ppr/FRiP,idr_frip_qc/rep1-pr/FRiP,idr_frip_qc/rep1-rep2/FRiP,idr_frip_qc/rep1-rep3/FRiP,idr_frip_qc/rep2-pr/FRiP,idr_frip_qc/rep2-rep3/FRiP,idr_frip_qc/rep3-pr/FRiP,idr_reproducibility_qc/N1,idr_reproducibility_qc/N2,idr_reproducibility_qc/N3,idr_reproducibility_qc/N_consv,idr_reproducibility_qc/N_opt,idr_reproducibility_qc/Np,idr_reproducibility_qc/Nt,idr_reproducibility_qc/consv_set,idr_reproducibility_qc/opt_set,idr_reproducibility_qc/reproducibility,idr_reproducibility_qc/rescue_ratio,idr_reproducibility_qc/self_consistency_ratio,nodup_flagstat_qc/rep1/diff_chroms,nodup_flagstat_qc/rep1/diff_chroms_qc_failed,nodup_flagstat_qc/rep1/duplicates,nodup_flagstat_qc/rep1/duplicates_qc_failed,nodup_flagstat_qc/rep1/mapped,nodup_flagstat_qc/rep1/mapped_pct,nodup_flagstat_qc/rep1/mapped_qc_failed,nodup_flagstat_qc/rep1/paired,nodup_flagstat_qc/rep1/paired_properly,nodup_flagstat_qc/rep1/paired_properly_pct,nodup_flagstat_qc/rep1/paired_properly_qc_failed,nodup_flagstat_qc/rep1/paired_qc_failed,nodup_flagstat_qc/rep1/read1,nodup_flagstat_qc/rep1/read1_qc_failed,nodup_flagstat_qc/rep1/read2,nodup_flagstat_qc/rep1/read2_qc_failed,nodup_flagstat_qc/rep1/singletons,nodup_flagstat_qc/

In [50]:
df.to_excel('/users/avsec//gdrive/projects/chipnexus/paper/tables/crispr-all-qc.xlsx')

## Processed files to export

In [8]:
# Extract processed files from the output directory
def processed_bigwig(root, strand):
    bigwigs = list((Path(root) / 'call-count_signal_track_pooled/execution').glob(f'*.{strand}.bigwig'))
    assert len(bigwigs) == 1
    return bigwigs[0]

def processed_narrowPeak(root):
    return Path(root) / 'call-reproducibility_idr/execution/optimal_peak.narrowPeak.gz'


def processed_all_files(root):
    return {'bigwig.positive': processed_bigwig(root, 'positive'),
            'bigwig.negative': processed_bigwig(root, 'negative'),
            'narrowPeak': processed_narrowPeak(root)
            }

In [9]:
processed_files = {sample: processed_all_files(config['workflowRoot'])
                   for sample,config in configs.items()}
df_processed_files = pd.DataFrame(processed_files).T
df_processed_files

,bigwig.negative,bigwig.positive,narrowPeak
wt.Nanog,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...
wt.Sox2,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...
mutant.sox2crispr1.Nanog,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...
mutant.sox2crispr1.Sox2,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...
mutant.nanogcrispr1.Nanog,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...
mutant.nanogcrispr1.Sox2,/oak/stanford/groups/...,/oak/stanford/groups/...,/oak/stanford/groups/...


In [14]:
# Molten version
dfm = df_processed_files.stack().reset_index(level=1)
dfm.columns = ['file_type', 'path']
df_fastq = pd.DataFrame([{'sample': sample, 'file_type': 'fastq', 'path': path}
                         for sample, paths in df.fastqs.items() for path in paths]).set_index('sample')
dfm = pd.concat([dfm, df_fastq], axis=0)
assert dfm.path.map(os.path.exists).all()  # All files exist

# Compute MD5 hashes
dfm['md5'] = Parallel(-1, verbose=10)(delayed(md5sum)(path) for path in dfm.path)

dfm

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  35 | elapsed:  2.2min remaining: 17.3min
[Parallel(n_jobs=-1)]: Done   8 out of  35 | elapsed:  2.6min remaining:  8.8min
[Parallel(n_jobs=-1)]: Done  12 out of  35 | elapsed:  3.0min remaining:  5.8min
[Parallel(n_jobs=-1)]: Done  16 out of  35 | elapsed:  3.5min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  20 out of  35 | elapsed: 23.0min remaining: 17.2min
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed: 25.8min remaining: 11.8min
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed: 32.9min remaining:  8.2min
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed: 39.9min remaining:  3.7min
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed: 41.3min finished


,file_type,path,md5
wt.Nanog,bigwig.negative,/oak/stanford/groups/...,8469fa39110bf17f56952...
wt.Nanog,bigwig.positive,/oak/stanford/groups/...,655bfde8d14216c738202...
wt.Nanog,narrowPeak,/oak/stanford/groups/...,5976e226bc57531ce6c4b...
...,...,...,...
mutant.nanogcrispr1.Sox2,fastq,/oak/stanford/groups/...,fcab2929517ff7aa8ff26...
mutant.nanogcrispr1.Sox2,fastq,/oak/stanford/groups/...,fb25f045c9c4aeed88ff5...
mutant.nanogcrispr1.Sox2,fastq,/oak/stanford/groups/...,2ec0aa7956893d2bf2c5b...


In [49]:
dfm.to_excel('/users/avsec//gdrive/projects/chipnexus/paper/tables/file-hashes.xlsx')

In [16]:
## Make sure the md5-hashes are the same as from md5sum for some files
for i in [0, -1]:
    outputs = !md5sum {dfm.iloc[i].path}
    md5sum2 = outputs[0].split(' ')[0]
    print(md5sum2, dfm.iloc[i].md5)
    assert md5sum2 == dfm.iloc[i].md5

8469fa39110bf17f56952987a70ef311 8469fa39110bf17f56952987a70ef311
2ec0aa7956893d2bf2c5b3c2615d1955 2ec0aa7956893d2bf2c5b3c2615d1955


### Add qc to zip files

In [17]:
# from zipfile import ZipFile

# # writing files to a zipfile 
# with ZipFile(f'{ddir}/gdata/qc-htmls.zip','w') as zf: 
#     for i,row in reps.iterrows():
#         zf.write(f"{row.QC_dir}/qc.html", arcname="qc." + row['id'] + ".html") 

In [18]:
# Copy qc
for sample, row in df.iterrows():
    !mkdir -p /srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/
    !cp {row['chip_nexus.report']} /srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/qc.html
    !chmod o+rx /srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/qc.html

In [25]:
dfo = []
for sample, row in df.iterrows():
    for j, fastq in enumerate(row.fastqs):
        dfo.append({'sample': sample,
                    'path': os.path.basename(fastq),
                    '#Rep-IDRpeaks (N1, N2, ..)': row[f'idr_reproducibility_qc/N{j+1}'],
                    '#IDR-optimal peaks (Np)': row[f'idr_reproducibility_qc/Np'],
                    'Unique deduped reads': row[f'pbc_qc/rep{j+1}/distinct_read_pairs'],
                    'QC report': f'http://mitra.stanford.edu/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/qc.html',
                    'md5': dfm[dfm.path == fastq].iloc[0].md5,
                    'idr_reproducibility_qc/rescue_ratio': row['idr_reproducibility_qc/rescue_ratio'],
                    'idr_frip_qc/ppr/FRiP': row['idr_frip_qc/ppr/FRiP'],
                    })
dfo = pd.DataFrame(dfo)

In [28]:
print(dfo.to_csv(index=False))

#IDR-optimal peaks (Np),"#Rep-IDRpeaks (N1, N2, ..)",QC report,Unique deduped reads,idr_frip_qc/ppr/FRiP,idr_reproducibility_qc/rescue_ratio,md5,path,sample
95658,44149,http://mitra.stanford.edu/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.nanogcrispr1.Sox2/qc.html,85074201,0.193377713437,1.1960688698,fcab2929517ff7aa8ff26952e4060179,mesc_nanogcrispr1_sox2_B07_nexus_1.fastq.gz,mutant.nanogcrispr1.Sox2
95658,59005,http://mitra.stanford.edu/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.nanogcrispr1.Sox2/qc.html,100162662,0.193377713437,1.1960688698,fb25f045c9c4aeed88ff529c6b4c38c8,mesc_nanogcrispr1_sox2_B09_nexus_1.fastq.gz,mutant.nanogcrispr1.Sox2
95658,52844,http://mitra.stanford.edu/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.nanogcrispr1.Sox2/qc.html,118872840,0.193377713437,1.1960688698,2ec0aa7956893d2bf2c5b3c2615d1955,mesc_nanogcrispr1_sox2_F10_nexus_1.fastq.gz,mutant.nanogcrispr1.Sox2



In [29]:
# Copy files to webserver
filenames = {'bigwig.negative': 'counts.neg.bw',
             'bigwig.positive': 'counts.pos.bw',
             'narrowPeak': 'optimal_peak.narrowPeak.gz'}

for sample, row in dfm[dfm.file_type != 'fastq'].iterrows():
    !mkdir -p /srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/
    fn = filenames[row['file_type']]
    file_path = f'/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/{sample}/{fn}'
    print(file_path)
    !cp {row['path']} {file_path}
    !chmod o+rx {file_path}

/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Nanog/counts.neg.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Nanog/counts.pos.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Nanog/optimal_peak.narrowPeak.gz
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Sox2/counts.neg.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Sox2/counts.pos.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/wt.Sox2/optimal_peak.narrowPeak.gz
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.sox2crispr1.Nanog/counts.neg.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.sox2crispr1.Nanog/counts.pos.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.sox2crispr1.Nanog/optimal_peak.narrowPeak.gz
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-nexus/crispr/mutant.sox2crispr1.Sox2/counts.neg.bw
/srv/www/kundaje/avsec/chipnexus/paper/data/chip-ne

### Generate Supplementary table 1

In [30]:
from basepair.exp.paper.config import data_sheet

In [37]:
reps = data_sheet()
reps = reps[(~reps['QC report'].isnull()) | (reps['Comments'] == 'Crispr samples')]
reps = reps[(reps['TF Name'] != 'control')]
reps['Rep Number'] = reps['Rep Number'].astype(int)
reps['ID'] = [f"{row['Data Type']}-{row['TF Name']}-{row['Rep Number']}" for i,row in reps.iterrows()]
reps = reps[reps['TF Name'].isin(['oct4', 'sox2', 'nanog', 'klf4'])]
reps = reps[~((reps['TF Name'] == 'klf4')&(reps['Data Type'] == 'chipseq'))]

In [36]:
new_reps

,Mnemonic,FTP Name,thenexus Name,Sample ID,Comments,Reference,Data Type,TF Name,Rep Number,Control Reps,QC report,Unique deduped reads,Held-out test,"#Rep-IDRpeaks (N1, N2, ..)",#IDR-optimal peaks (Np),Md5-hash-FASTQ,Md5-hash-IDRoptimal,Md5-bigwigs,idr_frip_qc/ppr/FRiP,idr_reproducibility_qc/rescue_ratio,Oak path bw neg,ID
49,N34,mesc_wt_nanog_nexus_1...,wt.Nanog,NaN,Crispr samples,mm10,chipnexus,nanog,1,NaN,http://mitra.stanford...,65M,False,66649.0,38127.0,ac80395cba9da08516f9d...,NaN,NaN,0.4199,2.755,NaN,chipnexus-nanog-1
50,N35,mesc_wt_nanog_nexus_1...,wt.Nanog,NaN,Crispr samples,mm10,chipnexus,nanog,2,NaN,http://mitra.stanford...,84M,False,16716.0,38127.0,126dac02c288489bff786...,NaN,NaN,0.4199,2.755,NaN,chipnexus-nanog-2
51,N36,mesc_wt_nanog_nexus_1...,wt.Nanog,NaN,Crispr samples,mm10,chipnexus,nanog,3,NaN,http://mitra.stanford...,148M,False,10644.0,38127.0,20cfef8acca36b46f641d...,NaN,NaN,0.4199,2.755,NaN,chipnexus-nanog-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,N48,mesc_nanogcrispr1_sox...,mutant.nanogcrispr1.Sox2,NaN,Crispr samples,mm10_nanogcrispr1,chipnexus,sox2,1,NaN,http://mitra.stanford...,85M,False,44149.0,95658.0,fcab2929517ff7aa8ff26...,NaN,NaN,0.1934,1.196,NaN,chipnexus-sox2-1
64,N49,mesc_nanogcrispr1_sox...,mutant.nanogcrispr1.Sox2,NaN,Crispr samples,mm10_nanogcrispr1,chipnexus,sox2,2,NaN,http://mitra.stanford...,100M,False,59005.0,95658.0,fb25f045c9c4aeed88ff5...,NaN,NaN,0.1934,1.196,NaN,chipnexus-sox2-2
65,N50,mesc_nanogcrispr1_sox...,mutant.nanogcrispr1.Sox2,NaN,Crispr samples,mm10_nanogcrispr1,chipnexus,sox2,3,NaN,http://mitra.stanford...,119M,False,52844.0,95658.0,2ec0aa7956893d2bf2c5b...,NaN,NaN,0.1934,1.196,NaN,chipnexus-sox2-3


In [43]:
# Add df_data
df_data = pd.read_excel('/users/avsec/gdrive/projects/chipnexus/paper/tables/data-sheet.qc.xlsx')
new_reps = reps[~reps.Mnemonic.isin(df_data.Mnemonic)]
new_reps['idr_reproducibility_qc/rescue_ratio'] = df['idr_reproducibility_qc/rescue_ratio'].loc[new_reps['thenexus Name']].values
new_reps['idr_frip_qc/ppr/FRiP'] = df['idr_frip_qc/ppr/FRiP'].loc[new_reps['thenexus Name']].values
new_reps['ID'] = new_reps['thenexus Name'] + '-' + reps['Rep Number'].astype(str)
df_data = pd.concat([df_data, new_reps[df_data.columns]], axis=0)

/users/avsec/bin/anaconda3/envs/chipnexus/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/users/avsec/bin/anaconda3/envs/chipnexus/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/users/avsec/bin/anaconda3/envs/chipnexus/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [33]:
df_data.to_excel('/users/avsec//gdrive/projects/chipnexus/paper/tables/data-sheet.qc.v2.xlsx')